In [22]:
import os
from pathlib import Path

from dotenv import load_dotenv
from nbiatoolkit.nbia import NBIAClient
from rich import print
from tcia_utils import nbia
from yaml import safe_load
import asyncio
load_dotenv()

NBIA_USERNAME = os.getenv("NBIA_USERNAME")
NBIA_PASSWORD = os.getenv("NBIA_PASSWORD")

nbia.getToken(NBIA_USERNAME, NBIA_PASSWORD)


2025-01-16 16:50:17,914:INFO:Success - Token saved to global api_call_headers variable and expires at 2025-01-16 18:50:17.914462


In [2]:
notebook_path = Path().cwd()
print(notebook_path)
yaml_file = notebook_path / 'nbia.yaml'

with yaml_file.open() as f:
    config = safe_load(f)
  
print(config)

/home/bioinf/bhklab/radiomics/repos/med-image_test-data

{
    'datasets': {
        'CC-Tumor-Heterogeneity': {'patients': ['CCTH-B11', 'CCTH-A01']},
        'Vestibular-Schwannoma-SEG': {'patients': ['VS-SEG-001', 'VS-SEG-002']},
        'NSCLC-Radiomics': {'patients': ['LUNG1-001', 'LUNG1-002']}
    }
}

In [20]:
datasets = []

for collection in config['datasets']:
    series = []
    for patient in config['datasets'][collection]['patients']:
        result = nbia.getSeries(patientId=patient)
        series.extend(result)
        break
        
    nbia.downloadSeries(series, as_zip=True)
    break


2025-01-16 16:46:28,018:INFO:Calling getSeries with parameters {'PatientID': 'CCTH-B11'}
2025-01-16 16:46:29,004:INFO:Downloading 19 out of 19 Series Instance UIDs (scans).
2025-01-16 16:46:29,005:INFO:Directory 'tciaDownload' created successfully.
2025-01-16 16:46:29,006:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.146654035817310821905606741421308676770
2025-01-16 16:47:22,438:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.95596873266224187066449317160430410682
2025-01-16 16:47:25,004:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.22534084187379624651252726645637819677
2025-01-16 16:48:16,906:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/g

KeyboardInterrupt: 

In [27]:
async def download_multiple_series():
    client = NBIAClient(NBIA_USERNAME, NBIA_PASSWORD)
    for collection in config['datasets']:
        series = []
        for patient in config['datasets'][collection]['patients']:
            params = {
                "PatientID": patient
            }
            result = await client.getSeries(params)
            series.extend(result)
            break
        series = series[:2]
        print(series)
        await client.download_series(series, as_zip=True)
        break

# get current event loop
loop = asyncio.get_event_loop()

# run until complete
loop.run_until_complete(download_multiple_series())

RuntimeError: This event loop is already running